<a href="https://colab.research.google.com/github/misabhishek/gcp-iam-recommender/blob/main/all_firewall_insights_in_an_org.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Problem Statement

Get all firewall insights in an org 

### Imports

In [ ]:
from google.colab import auth
import json
import subprocess

import concurrent
import logging

logging.basicConfig(format="%(levelname)s[%(asctime)s]:%(message)s")

### Lets authenticate

In [ ]:
auth.authenticate_user()
print('Authenticated')

Authenticated


### Helper functions

In [ ]:
def execute_command(command):
  return json.loads(subprocess.check_output(filter(lambda x: x, command.split(" "))).decode("utf-8"))

### Enter your org details

In [ ]:
organization = "" # organizations/[ORGANIZATION-ID] 
billing_project_id = "" # Billing project

### Get All the projects

In [ ]:
get_all_projects_command = f"""gcloud asset search-all-resources \
                               --asset-types=cloudresourcemanager.googleapis.com/Project \
                               --scope={organization} --format=json --project={billing_project_id}"""

In [ ]:
def get_all_projects():
  projects = execute_command(get_all_projects_command)
  return [p["additionalAttributes"]["projectId"] for p in projects]

project_ids = get_all_projects()

print("project-ids\n\n", "\n".join(project_ids[:10]))

### Get All Firewall insights

In [ ]:
firewall_insight_command = """gcloud beta recommender insights list \
                           --project={} \
                           --location=global --insight-type=google.compute.firewall.Insight \
                           --format=json --billing-project=""" + billing_project_id

In [ ]:
def get_all_firewall_insights():
  def get_insights(project_id):
    try:
      return {"project_id":project_id,
              "insights": execute_command(firewall_insight_command.format(project_id))}
    except:
      logging.warning(f"You don't have permissions to access project:`{project_id}`")
      return {"project_id": project_id,
              "insights": []}

  with concurrent.futures.ThreadPoolExecutor(max_workers=100) as executor:
    insights = {"all_insights": list(executor.map(get_insights, project_ids))}
  
  return insights

In [ ]:
all_firewall_insights = get_all_firewall_insights()